In [7]:
from crbranch.branch import Branch
from crbranch.crbranch import CrBranch
from crbranch.footpath import Footpath
import osmnx as ox
import shapely
import geopandas as gpd

import folium
map = folium.Map(location=[45.77346,3.09125], zoom_start=19)

def addGeomToMap(geom, color="#0000FF"):
    folium.GeoJson(shapely.to_geojson(ox.projection.project_geometry(geom, crs="EPSG:3857", to_crs="EPSG:4326")[0]), style_function = lambda feature: {'color':color}).add_to(map)

def addDFToMap(df, color="#0000FF"):
    for geom in df.geometry:
        addGeomToMap(geom, color)

In [8]:
# Load data
crbranch = CrBranch()
crbranch.load_data(osm_file = "data/example_clermont_18_07_2023.osm", lat=45.77349, lng=3.09111)
nodes = [21486411, 724112978, 4734568822, 8706544706, 566967069, 25891548, 21647182, 724111817]
branch = Branch([Footpath(Footpath.Side.LEFT, nodes), Footpath(Footpath.Side.RIGHT, nodes)])
osm_data = ox.projection.project_gdf(crbranch.osm_data, to_crs="EPSG:3857")

In [9]:
# Geometries of the way
left_geom = branch.footpaths[0].get_geometry(crbranch.G)
right_geom = branch.footpaths[-1].get_geometry(crbranch.G)

# Buffer for each side
distance = 20
left_buffer = shapely.buffer(left_geom, distance, single_sided=True)
right_buffer = shapely.buffer(right_geom, -distance, single_sided=True)

In [10]:
# Filter OSM datas by wanted objects
buildings = osm_data.query("building.notna() and building != 'no'")
pois = osm_data.query("`addr:housenumber`.notna() or amenity in ('restaurant','fast_food','bank','bar','pharmacy') or highway in ('bus_stop') or shop.notna() or leisure.notna() or office.notna() or tourism.notna()")
pois = pois[pois.disjoint(left_geom)][pois.disjoint(right_geom)]

# Get crosswalks
crosswalks = osm_data.query("((crossing.notna() and crossing != 'no') or (highway.notna() and highway=='crossing')) and (foot.isna() or foot != 'no')")

/opt/homebrew/Caskroom/miniforge/base/envs/gis/lib/python3.11/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [16]:
# Compute datas on the sides of the way
left_buildings = buildings[buildings.intersects(left_buffer)]
right_buildings = buildings[buildings.intersects(right_buffer)]
left_pois = pois[pois.intersects(left_buffer)]
right_pois = pois[pois.intersects(right_buffer)]

# Compute crosswalks for each sides
left_crosswalks = crosswalks[crosswalks.intersects(shapely.buffer(left_geom,0.5))]
right_crosswalks = crosswalks[crosswalks.intersects(shapely.buffer(right_geom,0.5))]

In [ ]:
# Search forking ways

In [17]:
map = folium.Map(location=[45.77346,3.09125], zoom_start=19)
addGeomToMap(left_buffer)
addDFToMap(left_buildings)
addDFToMap(left_pois)
addGeomToMap(left_geom, "#FF0000")
addDFToMap(left_crosswalks)
map

/opt/homebrew/Caskroom/miniforge/base/envs/gis/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/opt/homebrew/Caskroom/miniforge/base/envs/gis/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/opt/homebrew/Caskroom/miniforge/base/envs/gis/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.